In [ ]:
# Parameters: 
politician_id <- "240"
aggregation <- "month" # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

## Using API

In [ ]:
base <- "193.175.238.88:8080/api/politicians/"
call1 <- paste(base,"all/", sep="")
get_politicians_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
call_chobs <- paste(base,"wikipedia/chobs/politicians/", politician_id, "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
chobs = as.data.frame(fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"), flatten = TRUE))

#extracting revisions, conflicts and undos via wikiwho API
base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"
wiki_id <- get_politicians_df$wp_ids[get_politicians_df$politician_id == politician_id]
wiki = as.data.frame(fromJSON(content(GET(paste(base_ww, wiki_id, "/", sep="")), "text", encoding="UTF-8"), flatten = TRUE))
wiki$undos <- wiki$editions.dels + wiki$editions.reins
wiki <- summarise(group_by(wiki, editions.year_month), undos = sum(undos),revisions = sum(editions.revisions), conflicts = sum(editions.conflicts))

#merged = merge(chobs, wiki, by.x='labels', by.y='editions.year_month')

In [ ]:
page_id<-get_politicians_df[politician_id,c('wp_ids')]
call_page<-paste("https://de.wikipedia.org/w/api.php?action=query&prop=info&pageids=",page_id,"&inprop=url&format=json", sep="")
page = as.data.frame(fromJSON(content(GET(call_page), "text", encoding="UTF-8"), flatten = TRUE))

In [ ]:
wiki_title=page[, 4]
call_views <- paste("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/all-agents/", wiki_title,"/", aggregation,"ly/20171101/20181101", sep="")
views = as.data.frame(fromJSON(content(GET(call_views), "text", encoding="UTF-8"), flatten = TRUE))

In [ ]:
views<-views[c("items.timestamp", "items.views")]
views$items.timestamp<-as.POSIXct(strptime(views[["items.timestamp"]], format='%Y%m%d%H'),tz = 'GMT')
wiki$editions.year_month<-as.POSIXct(strptime(wiki[["editions.year_month"]], format='%Y-%m-%d'),tz = 'GMT')
chobs$labels<-as.POSIXct(chobs[["labels"]],tz = 'GMT')

In [ ]:
wiki_data = merge( views, wiki, by.y='editions.year_month', by.x='items.timestamp', all.x = TRUE)
merged = merge(chobs, wiki_data, by.x='labels', by.y='items.timestamp', all.x = TRUE)

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  stat_smooth(aes(y = values, color="Change Objects", group=response_type), se = F, method = "lm", formula = y ~ poly(x, 8)) +
  stat_smooth(aes(y = revisions, color="Revisions", group=response_type), se = F, method = "lm", formula = y ~ poly(x, 8)) +
   stat_smooth(aes(y = items.views/1000, color="Views", group=response_type), se = F, method = "lm", formula = y ~ poly(x, 8))+
    scale_y_continuous(sec.axis = sec_axis(~.*1,name = "Views",labels = function(b) { paste0(b, "K")}))+
  labs(title = "Wikipedia Activity", x = "", y = "Change Objects and Revisions") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month")

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  stat_smooth(aes(y = undos, color="Undos", group=response_type), se = F, method = "lm", formula = y ~ poly(x, 8)) +
  stat_smooth(aes(y = conflicts, color="Conflicts", group=response_type), se = F, method = "lm", formula = y ~ poly(x, 8)) +
  labs(title = "Wikipedia Conflicts", x = "", y = "undos and conflicts") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month")+ scale_y_continuous(labels = scales::number_format(big.mark = ""))